In [1]:
import os
import sys
import cv2
import torch
import requests
import numpy as np
from tqdm import tqdm
from pathlib import Path
from PIL import Image
%matplotlib inline

from config import(
    DIRECTORY_MODEL, 
    DETECTION_MODEL,
    CLASSES_DETECTION,
    CLASSES_FILTERED
)

In [2]:
class SealDetection:
    '''
    Load custom model YoloV5
    for detection seal container number
    '''
    def __init__(self, model_name):
        self.model_name = model_name
        self.device     = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model_path = os.path.join(DIRECTORY_MODEL, DETECTION_MODEL[self.model_name]['filename'])
        self.__check_model()
        self.model      = self.__load_model(self.model_path)
        
    @staticmethod
    def __load_model(model_path):
        try: model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
        except: sys.exit('Error load model')
        return model
            
    def __check_model(self):
        '''
        Checking model in model_path
        download model if file not found
        '''
        Path(DIRECTORY_MODEL).mkdir(parents=True, exist_ok=True)
        if not os.path.isfile(self.model_path):
            print(f'Downloading {self.model_name} detection model, please wait.')
            response = requests.get(DETECTION_MODEL[self.model_name]['url'], stream=True)
            
            progress = tqdm(response.iter_content(1024), 
                        f'Downloading {DETECTION_MODEL[self.model_name]["filename"]}', 
                        total=DETECTION_MODEL[self.model_name]['file_size'], unit='B', 
                        unit_scale=True, unit_divisor=1024)
            with open(self.model_path, 'wb') as f:
                for data in progress:
                    f.write(data)
                    progress.update(len(data))
                print(f'Done downloaded {DETECTION_MODEL[self.model_name]["filename"]} detection model.')
        else:
            print(f'Load {DETECTION_MODEL[self.model_name]["filename"]} detection model.')

    @staticmethod
    def extract_result(results, min_confidence=0.0):
        '''
        Format result([tensor([[151.13147, 407.76913, 245.91382, 454.27802,   0.89075,   0.00000]])])
        Filter min confidence prediction and classes id/name
        Cropped image and get index max value confidence lavel
        Args:
            result(models.common.Detections): result detection YoloV5
            min_confidence(float): minimal confidence detection in range 0-1
        Return:
            result(dict): {
                casess:[{
                    confidence(float): confidence,
                    bbox(list) : [x_min, y_min, x_max, y_max]
                }]
            }
        '''
        results_format  = results.xyxy
        results_filter =  dict({i:list() for i in CLASSES_FILTERED})
        
        if len(results_format[0]) >= 1:
            for i in range(len(results_format[0])):
                classes_name    = CLASSES_DETECTION[int(results_format[0][i][-1])]
                confidence      = float(results_format[0][i][-2])
                # print(classes_name, confidence)
                if classes_name in CLASSES_FILTERED and confidence >= min_confidence:
                    x_min, y_min = int(results_format[0][i][0]), int(results_format[0][i][1])
                    x_max, y_max = int(results_format[0][i][2]), int(results_format[0][i][3])
                    results_filter[classes_name].append(
                        {'confidence': round(confidence,2), 
                        'bbox':[x_min, y_min, x_max, y_max]}
                    )
        # Delete key if detection null
        for i in CLASSES_FILTERED:results_filter.__delitem__(i) if not results_filter[i] else None

        return results_filter
    
    def detection(self, image, size=None):
        '''
        Prediction image object detectionn YoloV5
        Args:
            image(numpy.ndarray) : image/frame
        Return:
            results_prediction(models.common.Detections) : results -> convert to (results.xyxy/resultsxywh)
        '''
        if size: results = self.model(image, size=size)
        else: results = self.model(image)
        return results

In [3]:

class MainProcess:
    '''
        Main process seal detection
    '''
    def __init__(self):
        self.seal_detection = SealDetection('seal_detection')

    def __detection(self, image, size, threshold):
        '''
            Detection object
            Args:
                image(np.array) : image for detection
                size(int)       : size image detection
                threshold(float): min confidence
            Return:
                result(dict): {
                casess:[{
                    confidence(float): confidence,
                    bbox(list) : [x_min, y_min, x_max, y_max]
                }]
            }
        '''
        results = self.seal_detection.\
                detection(image=image, size=size)
        result  = self.seal_detection.\
                extract_result(results=results, min_confidence=threshold)
        return result
        
    def main(self, image, size, threshold):
        # Detection seal
        result = self.__detection(image, size=size, threshold=threshold)
        if not result: print(f'Seal not found'); return list()
        else: print('Seal Found')
        print(result)
        # Extract result to list
        result_list = [[[x['bbox'], key,  x['confidence']] for x in value]\
                      for key, value in result.items()][0]

        return result_list

In [173]:
def draw_rectangle_list(image, result_list, resize=100):
    '''
    Draw bounding box, label and clases detection image
    Args:
        img(numpy.ndarray) : image/frame
        result(list) : [[x_min, y_min, x_max, y_max, classes_name, confidence]]
    return : 
        image(numpy.ndarray)
    '''
    if len(result_list):
        for i in result_list:
            x_min, x_max = i[0][0], i[0][2]
            y_min, y_max = i[0][1], i[0][3]
            classes_name = i[1]
            confidence   = int(i[2]*100)
            color 		 = (255, 117, 0)
            # Draw rectangle
            
            # Add label
            if classes_name == 'seal':
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)
                cv2.rectangle(image, (x_min, y_min-35), (x_min+150, y_min), color, cv2.FILLED)
                cv2.putText(image, f'{classes_name} found'.upper(), (x_min-2,y_min-12), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 1)
            elif len(classes_name) > 4:
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)
                cv2.rectangle(image, (x_min, y_min-50), (x_min+310, y_min), color, cv2.FILLED)
                cv2.putText(image, f'{classes_name}'.upper(), (x_min+10,y_min-12), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            else:
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)
                cv2.rectangle(image, (x_min, y_max+40), (x_min+120, y_max), color, cv2.FILLED)
                cv2.putText(image, f'{classes_name}'.upper(), (x_min+10,y_max+30), cv2.FONT_HERSHEY_COMPLEX, 1.2, (255, 255, 255), 2)
    width   = int(image.shape[1] * resize / 100)
    height  = int(image.shape[0] * resize / 100)
    # resize image
    resized = cv2.resize(image, (width, height), interpolation = cv2.INTER_AREA)
    return resized

In [5]:
process = MainProcess()

Load seal_detection_v1_2.pt detection model.


Using cache found in /home/smartcam/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-2-28 torch 1.8.0 CUDA:0 (NVIDIA Tegra X1, 3964MiB)

Fusing layers... 
Model Summary: 232 layers, 7249215 parameters, 0 gradients
Adding AutoShape... 


In [210]:
image   = cv2.imread('./images/new2/images26.jpeg')
results = process.main(image, size=None, threshold=0.0)
drawed  = draw_rectangle_list(image, results, resize=100)

Image.fromarray(cv2.cvtColor(drawed, cv2.COLOR_BGR2RGB))

Seal Found
{'seal': [{'confidence': 0.94, 'bbox': [596, 824, 618, 882]}]}


[[[596, 824, 618, 882], 'seal', 0.94]]

In [211]:
results.append([[561, 201, 909, 306], 'CBHU7008108', 0.8939186930656433])
results.append([[696, 258, 814, 355], '45G1', 0.8484043478965759])

# [[561, 201, 909, 306, 'container_number', 0.6439760327339172], [696, 258, 814, 355, 'iso_code', 0.6439760327339172]]

In [213]:
# name = 'images3.jpeg'
# image   = cv2.imread(f'./images/new2/{name}')
drawed  = draw_rectangle_list(image, results, resize=100)

Image.fromarray(cv2.cvtColor(drawed, cv2.COLOR_BGR2RGB))
cv2.imwrite(f'./images/new2/results/images26.jpeg', drawed)

True

In [68]:
import glob

image_list = glob.glob('./images/new/*.jpeg')
for i in image_list:
    image = cv2.imread(i)
    print(i)
    results = process.main(image, size=None, threshold=0.78)
    drawed  = draw_rectangle_list(image, results, resize=40)
    cv2.imwrite(f'./images/new/results/{i.split("/")[-1]}', drawed)
    # break

./images/new/images 6.jpeg
Seal Found
{'seal': [{'confidence': 0.88, 'bbox': [829, 843, 886, 974]}]}
./images/new/images 13.jpeg
Seal Found
{'seal': [{'confidence': 0.94, 'bbox': [741, 833, 793, 955]}]}
./images/new/images 9.jpeg
Seal not found
./images/new/images 4.jpeg
Seal Found
{'seal': [{'confidence': 0.94, 'bbox': [785, 778, 832, 897]}, {'confidence': 0.82, 'bbox': [148, 58, 167, 107]}]}
./images/new/images 12.jpeg
Seal Found
{'seal': [{'confidence': 0.9, 'bbox': [761, 665, 822, 782]}]}
./images/new/images 5.jpeg
Seal not found
./images/new/images 15.jpeg
Seal not found
./images/new/images 7.jpeg
Seal not found
./images/new/images 14.jpeg
Seal Found
{'seal': [{'confidence': 0.91, 'bbox': [767, 667, 814, 778]}]}
./images/new/images 10.jpeg
Seal Found
{'seal': [{'confidence': 0.94, 'bbox': [670, 1176, 771, 1271]}]}
./images/new/images 8.jpeg
Seal not found
./images/new/images 11.jpeg
Seal not found
./images/new/images 1.jpeg
Seal Found
{'seal': [{'confidence': 0.93, 'bbox': [855, 7

In [ ]:
result = {'seal': [{'confidence': 0.93, 'bbox': [2324, 2400, 2009, 2197]}, {'confidence': 0.85, 'bbox': [2424, 2511, 2302, 2514]}]}

result_list = [[[x['bbox'], key,  x['confidence']] for x in value]\
                      for key, value in result.items()][0]
result_list